In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_table('/kaggle/input/sentiment-analysis-on-movie-reviews/train.tsv.zip')
train.head()
# sentiment 1 약간 부정



In [ ]:
train['Sentiment'].value_counts()

In [ ]:
train['Phrase'][0]

In [ ]:
test= pd.read_table('/kaggle/input/sentiment-analysis-on-movie-reviews/test.tsv.zip')
test.head()

In [ ]:
from keras.preprocessing.text import Tokenizer
tk = Tokenizer()

In [ ]:
tk.fit_on_texts(list(train['Phrase']) + list(test['Phrase']))

In [ ]:
tk.word_index

In [ ]:
train_text = tk.texts_to_sequences(train['Phrase']) 
test_text = tk.texts_to_sequences(test['Phrase'])

In [ ]:
pd.Series(train_text).apply(len)

In [ ]:
# 그림 - 
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize = (20, 12))
sns.distplot(pd.Series(train_text).apply(len))

In [ ]:
# 그림 - 
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize = (20, 12))
sns.distplot(pd.Series(test_text).apply(len))

In [ ]:
from keras.preprocessing.sequence import pad_sequences 

In [ ]:
pad_train = pad_sequences(train_text, maxlen= 30) # 너무 긴 거까지 고려하면 과 적합
pad_test = pad_sequences(test_text, maxlen= 30)

In [ ]:
from sklearn.model_selection import train_test_split # 데이터셋 세분화

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(pad_train, train['Sentiment'], random_state=2020, test_size = 0.15, stratify=train['Sentiment'])

In [ ]:
# 딥러닝 모델을 쌓는 방법은 2가지
# 스미싱 텍스트는 스미싱 키워드가 얼마나 들어가 있나가 중요해서 문맥을 고려하는 모델보다 아래 모델
# 1. 순서대로 쌓는 , 
# 1) 불러오기
from keras import Sequential
from keras.layers import Dense, Embedding, Flatten # 가장 많이쓰는 층 #텍스트에서는 embedding #딥러닝에서는 구조 중요, 차원맞춰줌

# 2) 모델쌓기, 선언
model = Sequential()     # 0도 추가해야함, 단어의 종류에서 그래서 +1
model.add(Embedding(len(tk.word_index)+1,1,input_length=30)) # 차원이 하나면 단어의 의미 분석이 제대로 안됨. 의미의 벡터를 늘려줄 필요가 있다. # 대회 시, 열심히 찾아나가야 되는 부분
# 1/13 예전보다 하나의 단어의미 자체가 중요해짐(=단어의 길이가 줄면) (보다 독립적). 이에, 차원의 갯수를 늘려주어야함
# 1/13 input_length는 100보다 커짐. 단어 하나하나가 쪼개지면서 많아짐

#embedding(단어의 의미를 학습한다), 각 데이터에 접근해 스미싱인가 아닌가 *단어*를 보고 판단해야함 
model.add(Flatten()) # 2차원 -> 1차원 
model.add(Dense(5, activation='softmax')) # 옵션2개 중 1개는 정답 클래스 갯수
# 1일때는 activation='sigmoid' -> 값이 0인 애들이 많이 나옴. 이에 앙상블이 잘 안됨.
model.compile(metrics = ['acc'], optimizer = 'adam', loss='sparse_categorical_crossentropy') # 3가지 옵션 넣어줄거임 * compile, 모델선언 완료
# 1일때는 binary_crossentropy
# sparse를 넣어주면 회귀로 인식하는 게 아닌 분류로 인식 + 코드의 간편성을 위해 원핫인코딩 안해도됨 
# 최적화함수는 최적의 지점을 가는 여러 방법, adam을 주로씀
# metrics = ['acc'] 정확도 까지 볼 수 있음

from keras.callbacks import EarlyStopping, ModelCheckpoint # 라이브러리는 항상 대문자 / 로스가 안좋아지면 과적합으로 판단해 자동으로 멈춰주는 /
# modelcheckpoint - 모델에서 가장 잘 나온 가중치를 저장하는 
es = EarlyStopping(patience=3) # 이미지 같은 경우는 평가로스가 많이 튀는 경우가 있어서 5 이상 정도로 
mc = ModelCheckpoint('best.h5', save_best_only=True) # h5는 가중치를 저장하는 #최적의 순간만 저장하겠다

# 3) 모델 학습
# model.fit(x_train, y_train, epochs=100, validation_split=0.1,batch_size=128, callbacks=[es,mc]) #callback #batch size는 32가 기본 / 텍스트는 배치가 크면 결과 값이 안나옴, 이미지는 배치가 크면 좋음 
# x, y 설정 / 학습 횟수 결정, epoch / 학습(90%), 평가(10%)


model.fit(x_train, y_train, epochs=100, validation_data=(x_valid, y_valid),batch_size=128, callbacks=[es,mc]) #callback #batch size는 32가 기본 / 텍스트는 배치가 크면 결과 값이 안나옴, 이미지는 배치가 크면 좋음 
# x, y 설정 / 학습 횟수 결정, epoch / 학습(90%), 평가(10%)


# 251553/251553 [==============================] - 11s 44us/step - loss: 2.1150e-05 - acc: 1.0000 - val_loss: 6.9053e-04 - val_acc: 0.9998
# -> random_State 변경하면 0.````3까지 나올 수 이음 

In [ ]:
# 최적 가중치 저장된 걸 가져와야함 
model.load_weights('best.h5')

In [ ]:
# predict
result = model.predict(pad_test, batch_size=128)
result
# 왼쪽 / 오른쪽(), 스미싱 확률

In [ ]:
result[1]

In [ ]:
result.argmax(1)
pd.DataFrame(result).idxmax(1)

In [ ]:
sub = pd.read_csv('/kaggle/input/sentiment-analysis-on-movie-reviews/sampleSubmission.csv')
sub.head()

In [ ]:
sub['Sentiment'] = result.argmax(1)

In [ ]:
sub.to_csv('baseline_평가셋 수정.csv', index=False)